In [1]:
import os
import sys
import numpy as np
import pandas as pd
import openmatrix as omx

In [2]:
os.chdir('../data/TransportationNetworks')
root = os.getcwd()
root

'c:\\Projects\\TrafficAssignmentApproaches\\data\\TransportationNetworks'

In [3]:
folders = [x for x in os.listdir(root)[1:] if os.path.isdir(os.path.join(root, x))]
folders

['.git',
 'Anaheim',
 'Austin',
 'Barcelona',
 'Berlin-Center',
 'Berlin-Friedrichshain',
 'Berlin-Mitte-Center',
 'Berlin-Mitte-Prenzlauerberg-Friedrichshain-Center',
 'Berlin-Prenzlauerberg-Center',
 'Berlin-Tiergarten',
 'Birmingham-England',
 'Braess',
 'chicago-regional',
 'Chicago-Sketch',
 'Eastern-Massachusetts',
 'GoldCoast',
 'Hessen-Asymmetric',
 'Munich',
 'Philadelphia',
 'SiouxFalls',
 'Sydney',
 'SymmetricaTestCase',
 'Terrassa-Asymmetric',
 'Winnipeg',
 'Winnipeg-Asymmetric',
 '_scripts']

In [4]:
# Function to import OMX matrices
def import_matrix(matfile):
    f = open(matfile, 'r')
    all_rows = f.read()
    blocks = all_rows.split('Origin')[1:]
    matrix = {}
    for k in range(len(blocks)):
        orig = blocks[k].split('\n')
        dests = orig[1:]
        orig=int(orig[0])

        d = [eval('{'+a.replace(';',',').replace(' ','') +'}') for a in dests]
        destinations = {}
        for i in d:
            destinations = {**destinations, **i}
        matrix[orig] = destinations
    zones = max(matrix.keys())
    mat = np.zeros((zones, zones))
    for i in range(zones):
        for j in range(zones):
            mat[i, j] = matrix.get(i+1,{}).get(j+1,0)
            
    return mat

In [ ]:
import_matrix(os.path.join(root, 'SiouxFalls', 'SiouxFalls' + '_trips.tntp'))

In [6]:
# Importing the networks into a Pandas dataframe consists of a single line of code
# but we can also make sure all headers are lower case and without trailing spaces
def import_net(f):
    netfile = os.path.join(root, f, f + '_net.tntp')
    net = pd.read_csv(netfile, skiprows=8, sep='\t')

    trimmed= [s.strip().lower() for s in net.columns]
    net.columns = trimmed

    # And drop the silly first andlast columns
    net.drop(['~', ';'], axis=1, inplace=True)
    return net

In [7]:
import_net('SiouxFalls')

,init_node,term_node,capacity,length,free_flow_time,b,power,speed,toll,link_type
0,1,2,25900.200640,6,6,0.15,4,0,0,1
1,1,3,23403.473190,4,4,0.15,4,0,0,1
2,2,1,25900.200640,6,6,0.15,4,0,0,1
3,2,6,4958.180928,5,5,0.15,4,0,0,1
4,3,1,23403.473190,4,4,0.15,4,0,0,1
...,...,...,...,...,...,...,...,...,...,...
71,23,22,5000.000000,4,4,0.15,4,0,0,1
72,23,24,5078.508436,2,2,0.15,4,0,0,1
73,24,13,5091.256152,4,4,0.15,4,0,0,1
74,24,21,4885.357564,3,3,0.15,4,0,0,1


In [9]:
def process_dataset(net_name): 
    mat = import_matrix(os.path.join(root, net_name, net_name + '_trips.tntp'))
    net = import_net(net_name)
    number_of_zones = mat.shape[0]
    number_of_nodes = net[['init_node', 'term_node']].max().max()
    number_of_links = net.shape[0]
    with open(os.path.join(root, net_name,  net_name + '_net.csv'), 'w') as f:
        f.write(f"# NUMBER OF ZONES:{number_of_zones},")
        f.write(f"NUMBER OF NODES:{number_of_nodes},")
        f.write(f"NUMBER OF LINKS:{number_of_links}")
        f.write("\n")

        net.to_csv(f, index=False)
    
    with open(os.path.join(root, net_name,  net_name + '_trips.csv'), 'w') as f:
        f.write(f"# NUMBER OF ZONES:{number_of_zones}")
        f.write("\n") 
        
        np.savetxt(f, mat, delimiter=',', fmt='%d')

In [10]:
process_dataset('Braess')

In [26]:
def process_dataset_single_od_pair(net_name, od_pair_index, flow_multiplier = 10): 
    mat = import_matrix(os.path.join(root, net_name, net_name + '_trips.tntp'))
    mask = np.ones(mat.shape[0], dtype=bool)
    mask[od_pair_index] = False
    mat[mask] = 0
    mat[od_pair_index] *= flow_multiplier

    net = import_net(net_name)
    number_of_zones = mat.shape[0]
    number_of_nodes = net[['init_node', 'term_node']].max().max()
    number_of_links = net.shape[0]

    output_dir = os.path.join(root, net_name + f"_SingleODPair_{od_pair_index}")
    os.makedirs(output_dir, exist_ok=True)  # This creates the directory if it doesn't exist

    with open(os.path.join(output_dir,  net_name + f"_SingleODPair_{od_pair_index}" + '_net.csv'), 'w') as f:
        f.write(f"# NUMBER OF ZONES:{number_of_zones},")
        f.write(f"NUMBER OF NODES:{number_of_nodes},")
        f.write(f"NUMBER OF LINKS:{number_of_links}")
        f.write("\n")

        net.to_csv(f, index=False)
    
    with open(os.path.join(output_dir,  net_name + f"_SingleODPair_{od_pair_index}" + '_trips.csv'), 'w') as f:
        f.write(f"# NUMBER OF ZONES:{number_of_zones}")
        f.write("\n") 
        
        np.savetxt(f, mat, delimiter=',', fmt='%d')

In [27]:
process_dataset_single_od_pair('SiouxFalls', 0)

In [20]:
mat = import_matrix(os.path.join(root, "SiouxFalls", "SiouxFalls" + '_trips.tntp'))

In [ ]:
mask = np.ones(mat.shape[0], dtype=bool)
mask[0] = False
mask

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()